In [1]:
%load_ext autoreload
%autoreload 2

In [243]:
import json
from src.coordinator import Coordinator
from src.utils.preprocessing import TextPreprocessor
import pandas as pd

In [244]:
coord = Coordinator()

In [245]:
dataset = pd.DataFrame()
for file_path in coord.data_raw.iterdir():
    if file_path.name.endswith('jsonl'):
        user = file_path.stem
        user_df = pd.read_json(file_path, lines=True)
        dataset = dataset.append(user_df, ignore_index=True)
        print(user, user_df.shape[0])
print('TOTAL:', dataset.shape[0])

KimKardashian 28394
jtimberlake 3317
britneyspears 5117
jimmyfallon 11624
shakira 5409
rihanna 9623
YouTube 17047
taylorswift13 516
Twitter 10694
cnnbrk 52562
ArianaGrande 43356
Cristiano 3251
TheEllenShow 21214
BarackObama 7057
narendramodi 23741
ladygaga 7614
justinbieber 19762
selenagomez 4275
realDonaldTrump 42281
katyperry 10098
TOTAL: 326952


In [246]:
dataset.head(3)

,tweet_id,user,time_epoch,tweet,n_likes,n_retweets,n_replies,n_emojis,quoted_tweet,hashtags,mentions
0,1249884667572105216,KimKardashian,1586830701,https://twitter.com/ilivetodance/status/12498...,35811,5113,515,3,"{'screen_name': 'iLivetoDance', 'name': 'Beat ...",[],[]
1,1249733419401183232,KimKardashian,1586794640,JUST RESTOCKED: the @SKIMS Waist Trainer — ava...,3850,166,340,0,"{'screen_name': None, 'name': None, 'text': ''...",[],[skims]
2,1249436159371067393,KimKardashian,1586723768,Happy Easter pic.twitter.com/CGracu9q6R,28243,791,398,0,"{'screen_name': None, 'name': None, 'text': ''...",[],[]


In [247]:
dataset['quoted_tweet_screen_name'] = dataset['quoted_tweet'].apply(lambda tweet: tweet['screen_name'])
dataset['quoted_tweet_name'] = dataset['quoted_tweet'].apply(lambda tweet: tweet['name'])
dataset['quoted_tweet_hashtags'] = dataset['quoted_tweet'].apply(lambda tweet: tweet['hashtags'])
dataset['quoted_tweet_mentions'] = dataset['quoted_tweet'].apply(lambda tweet: tweet['mentions'])
dataset['quoted_tweet_n_emojis'] = dataset['quoted_tweet'].apply(lambda tweet: tweet['n_emojis'])
dataset['quoted_tweet'] = dataset['quoted_tweet'].apply(lambda tweet: tweet['text'])

In [248]:
dataset.head(3)

,tweet_id,user,time_epoch,tweet,n_likes,n_retweets,n_replies,n_emojis,quoted_tweet,hashtags,mentions,quoted_tweet_screen_name,quoted_tweet_name,quoted_tweet_hashtags,quoted_tweet_mentions,quoted_tweet_n_emojis
0,1249884667572105216,KimKardashian,1586830701,https://twitter.com/ilivetodance/status/12498...,35811,5113,515,3,@kourtneykardash VS. @KimKardashian \n\n@khloe...,[],[],iLivetoDance,Beat Killa,"[RRRemakes, Kuwtk, DontEVER, WhatOnEarth, True...","[kourtneykardash, KimKardashian, khloekardashi...",3
1,1249733419401183232,KimKardashian,1586794640,JUST RESTOCKED: the @SKIMS Waist Trainer — ava...,3850,166,340,0,,[],[skims],None,None,[],[],0
2,1249436159371067393,KimKardashian,1586723768,Happy Easter pic.twitter.com/CGracu9q6R,28243,791,398,0,,[],[],None,None,[],[],0


In [249]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326952 entries, 0 to 326951
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   tweet_id                  326952 non-null  int64 
 1   user                      326952 non-null  object
 2   time_epoch                326952 non-null  int64 
 3   tweet                     326952 non-null  object
 4   n_likes                   326952 non-null  int64 
 5   n_retweets                326952 non-null  int64 
 6   n_replies                 326952 non-null  int64 
 7   n_emojis                  326952 non-null  int64 
 8   quoted_tweet              326952 non-null  object
 9   hashtags                  326952 non-null  object
 10  mentions                  326952 non-null  object
 11  quoted_tweet_screen_name  10457 non-null   object
 12  quoted_tweet_name         10412 non-null   object
 13  quoted_tweet_hashtags     326952 non-null  object
 14  quot

## 1. Remove all links and pictures in a tweet

In [250]:
preprocessor = TextPreprocessor()

In [253]:
dataset['tweet'].apply(lambda tweet: len(preprocessor.url_rgx.findall(tweet))).value_counts()

0    186504
1    138424
2      1901
3       106
4        14
5         3
Name: tweet, dtype: int64

In [254]:
dataset['tweet'].apply(lambda tweet: len(preprocessor.pic_rgx.findall(tweet))).value_counts()

0    263902
1     63045
2         5
Name: tweet, dtype: int64

In [255]:
dataset['quoted_tweet'].apply(lambda tweet: len(preprocessor.url_rgx.findall(tweet))).value_counts()

0    324446
1      2450
2        53
5         2
3         1
Name: quoted_tweet, dtype: int64

In [256]:
dataset['quoted_tweet'].apply(lambda tweet: len(preprocessor.pic_rgx.findall(tweet))).value_counts()

0    322464
1      4487
2         1
Name: quoted_tweet, dtype: int64

**Drop links from tweet**

In [257]:
dataset['tweet'] = dataset['tweet'].apply(lambda tweet: preprocessor.clean_links(tweet))
dataset['tweet'].apply(lambda tweet: len(preprocessor.pic_rgx.findall(tweet))).value_counts(), \
dataset['tweet'].apply(lambda tweet: len(preprocessor.url_rgx.findall(tweet))).value_counts()

(0    326952
 Name: tweet, dtype: int64,
 0    326952
 Name: tweet, dtype: int64)

**Drop links from quoted tweets**

In [258]:
dataset['quoted_tweet'] = dataset['quoted_tweet'].apply(lambda tweet: preprocessor.clean_links(tweet))
dataset['quoted_tweet'].apply(lambda tweet: len(preprocessor.pic_rgx.findall(tweet))).value_counts(), \
dataset['quoted_tweet'].apply(lambda tweet: len(preprocessor.url_rgx.findall(tweet))).value_counts()

(0    326952
 Name: quoted_tweet, dtype: int64,
 0    326952
 Name: quoted_tweet, dtype: int64)

## 2. Drop tweets with less than 50 characters

**Number of tweets with less than 50 characters and more than 50 characters respectively**

In [259]:
dataset[dataset['tweet'].apply(lambda tweet: sum(len(token) for token in tweet.split()) < 50)].shape[0]

102697

In [260]:
dataset = dataset[dataset['tweet'].apply(lambda tweet: sum(len(token) for token in tweet.split()) >= 50)]
dataset.shape[0]

224255

In [261]:
dataset.groupby('user').size()

user
ArianaGrande       21810
BarackObama         6355
Cristiano           2014
KimKardashian      15125
TheEllenShow       16252
Twitter             1834
YouTube             9275
britneyspears       3154
cnnbrk             47489
jimmyfallon         8099
jtimberlake         2195
justinbieber        9210
katyperry           6416
ladygaga            5448
narendramodi       21574
realDonaldTrump    36170
rihanna             4887
selenagomez         2276
shakira             4384
taylorswift13        288
dtype: int64

In [262]:
dataset.head(3)

,tweet_id,user,time_epoch,tweet,n_likes,n_retweets,n_replies,n_emojis,quoted_tweet,hashtags,mentions,quoted_tweet_screen_name,quoted_tweet_name,quoted_tweet_hashtags,quoted_tweet_mentions,quoted_tweet_n_emojis
1,1249733419401183232,KimKardashian,1586794640,JUST RESTOCKED: the @SKIMS Waist Trainer — ava...,3850,166,340,0,,[],[skims],None,None,[],[],0
7,1249338384847220738,KimKardashian,1586700457,My sweet baby True. Happy 2nd Birthday! I wish...,30039,1075,174,2,,[],[],None,None,[],[],0
9,1249080285972410368,KimKardashian,1586638921,Does this ever happen to anyone else where you...,23384,797,2060,0,,[],[],None,None,[],[],0


In [264]:
dataset.to_json(coord.data_interim / 'dataset_v1.jsonl', lines=True, orient='records')

## 3. Preprocessing demonstraion on a few examples

In [291]:
preprocessor = TextPreprocessor()

In [293]:
examples = dataset['tweet'].iloc[:10].values
examples

array(['JUST RESTOCKED: the @SKIMS Waist Trainer — available now in Clay and Onyx and in sizes XXS - 4X. Shop now at     and enjoy free shipping on domestic orders over $75.',
       'My sweet baby True. Happy 2nd Birthday! I wish we could all be there with you to celebrate you today! I love you so much precious girl! Looking through pictures to post made me so happy, this cousin bond is so special and will last forever',
       'Does this ever happen to anyone else where your text messages on your iPhone are all blank and looks like you have no conversations going? How do we fix this without turning off and on your phone!',
       'One of my favorite bottles we designed!  We picked this shape bottle because each point of the bottle represents each child my mother has had. KKW X KRIS',
       'Hey guys!!! If you missed my IG LIVE with my attorney @JessicaJackson at 3PM on her page, you can now watch it on my IGTV    …',
       'There’s 2.2 mil people in prisons & jails right now who ca

In [294]:
for example in examples:
    print(example)
    print(preprocessor(example))
    print()

JUST RESTOCKED: the @SKIMS Waist Trainer — available now in Clay and Onyx and in sizes XXS - 4X. Shop now at     and enjoy free shipping on domestic orders over $75.
['restock', 'skims', 'waist', 'trainer', 'clay', 'onyx', 'size', 'xx', 'shop', 'enjoy', 'free', 'shipping', 'domestic', 'order']

My sweet baby True. Happy 2nd Birthday! I wish we could all be there with you to celebrate you today! I love you so much precious girl! Looking through pictures to post made me so happy, this cousin bond is so special and will last forever
['sweet', 'baby', 'true', 'happy', 'birthday', 'celebrate', 'today', 'love', 'precious', 'girl', 'picture', 'post', 'happy', 'cousin', 'bond', 'special']

Does this ever happen to anyone else where your text messages on your iPhone are all blank and looks like you have no conversations going? How do we fix this without turning off and on your phone!
['happen', 'text', 'message', 'iphone', 'blank', 'conversation', 'turn', 'phone']

One of my favorite bottles we